## 모듈 불러오기

In [105]:
import pandas as pd
import numpy as np
import re
import pickle
from tqdm import tqdm
import pymysql
from konlpy.tag import Mecab
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, accuracy_score
from gensim import corpora
from gensim.models.fasttext import FastText

In [33]:
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import *
from keras.preprocessing.sequence import pad_sequences

### Train SET : gcp_train_set.csv

In [8]:
df = pd.read_csv('./train/gcp_train_set.csv',encoding='utf-8-sig')
df

,video_id,title,channel_id,main_cate1,sub_cate1_1,sub_cate2_1,confidence1,main_cate2,sub_cate1_2,sub_cate2_2,confidence2,main_cate3,sub_cate1_3,sub_cate2_3,confidence3
0,_-_EByBFBsc,[오토캐드(Autocad) 기초 3D 26강] 경로 돌출(Sweep) 명령어 사용 방법,UCpOYwX9ebEnZ7b4UKTiLDCQ,Computers & Electronics,CAD & CAM,NaN,0.98,Arts & Entertainment,Visual Art & Design,Design,0.96,Science,Engineering & Technology,NaN,0.94
1,_-_RRUatHAU,[타로 / 연애 / 심리] 나는 누구와 사랑하게 될까?,UCQb1gwu_QKCrgegVx5nlEfw,People & Society,Religion & Belief,NaN,0.99,Arts & Entertainment,Offbeat,NaN,0.50,NaN,NaN,NaN,NaN
2,_--c_UgDDsk,갓세븐 제이비랑 울면서 영통한 썰 푼다 (GOT7 JB),UC2fqjCRQDGnl9BJ4NOfl2dA,Arts & Entertainment,Music & Audio,NaN,0.51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,_--MzCpmzE4,Black Bean Noodle Song | 창작동요 | 쥬니버 동요,UCAqJaUnVgAAeW0RDblGeY3w,People & Society,Kids & Teens,Children's Interests,0.96,Arts & Entertainment,Music & Audio,NaN,0.93,NaN,NaN,NaN,NaN
4,_--NaWhpU6E,[방탄소년단 슈가] 이럴슈가..,UCAKMhhj9qmldqJNWhQCuFMQ,Arts & Entertainment,Music & Audio,NaN,0.85,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
563028,ZZzyk-FkjlE,미니어쳐 빈티지 케이크 만들기/ miniature vintage cake (쪼끄맹 ...,UCVYfq00z2b38Z1EK9BMwsWg,Food & Drink,Food,NaN,0.64,Hobbies & Leisure,NaN,NaN,0.55,NaN,NaN,NaN,NaN
563029,ZzzyL2yM30Y,"레깅스 입고, 뛰어다니는 아줌마 Vlog",UCLpy1oRty_s6HxGqsj-Okzg,Shopping,Apparel,NaN,0.63,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
563030,ZZzYtQSwo6g,자동 수익으로 월 천만원? 유료 강의에 130만원 써 봤습니다. 돈 버는 게 그렇게...,UC3D7sMbh96dMUmaf8m-bKJA,Business & Industrial,Small Business,MLM & Business Opportunities,0.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
563031,ZzZyzYp8m9w,"[직캠] 191127 지창욱 베트남 인천공항 입국 (jichangwook,池昌旭) ...",UCAnRSFRuxYU087HNOGta3BA,Travel,Air Travel,NaN,0.99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
df['main_cate1_2'] = df['main_cate1'] + "/" + df['sub_cate1_1'] #Main 카테고리 + Sub 카테고리
df['main_cate1_3'] = df['main_cate1_2'] + "/" + df['sub_cate2_1'] #Main 카테고리 + Sub 카테고리 + Sub2 카테고리

In [10]:
df['main_cate2_2'] = df['main_cate2'] + "/" + df['sub_cate1_2']
df['main_cate2_3'] = df['main_cate2_2'] + "/" + df['sub_cate2_2']
df['main_cate3_2'] = df['main_cate3'] + "/" + df['sub_cate1_3']
df['main_cate3_3'] = df['main_cate3_2'] + "/" + df['sub_cate2_3']

### 학습을 할 카테고리로 재분류

In [12]:
df_train = df[['video_id','title','main_cate1','main_cate1_2','main_cate1_3','main_cate2','main_cate2_2','main_cate2_3','main_cate3','main_cate3_2','main_cate3_3']]

In [13]:
df_train

,video_id,title,main_cate1,main_cate1_2,main_cate1_3,main_cate2,main_cate2_2,main_cate2_3,main_cate3,main_cate3_2,main_cate3_3
0,_-_EByBFBsc,[오토캐드(Autocad) 기초 3D 26강] 경로 돌출(Sweep) 명령어 사용 방법,Computers & Electronics,Computers & Electronics/CAD & CAM,NaN,Arts & Entertainment,Arts & Entertainment/Visual Art & Design,Arts & Entertainment/Visual Art & Design/Design,Science,Science/Engineering & Technology,NaN
1,_-_RRUatHAU,[타로 / 연애 / 심리] 나는 누구와 사랑하게 될까?,People & Society,People & Society/Religion & Belief,NaN,Arts & Entertainment,Arts & Entertainment/Offbeat,NaN,NaN,NaN,NaN
2,_--c_UgDDsk,갓세븐 제이비랑 울면서 영통한 썰 푼다 (GOT7 JB),Arts & Entertainment,Arts & Entertainment/Music & Audio,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,_--MzCpmzE4,Black Bean Noodle Song | 창작동요 | 쥬니버 동요,People & Society,People & Society/Kids & Teens,People & Society/Kids & Teens/Children's Inter...,Arts & Entertainment,Arts & Entertainment/Music & Audio,NaN,NaN,NaN,NaN
4,_--NaWhpU6E,[방탄소년단 슈가] 이럴슈가..,Arts & Entertainment,Arts & Entertainment/Music & Audio,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
563028,ZZzyk-FkjlE,미니어쳐 빈티지 케이크 만들기/ miniature vintage cake (쪼끄맹 ...,Food & Drink,Food & Drink/Food,NaN,Hobbies & Leisure,NaN,NaN,NaN,NaN,NaN
563029,ZzzyL2yM30Y,"레깅스 입고, 뛰어다니는 아줌마 Vlog",Shopping,Shopping/Apparel,NaN,NaN,NaN,NaN,NaN,NaN,NaN
563030,ZZzYtQSwo6g,자동 수익으로 월 천만원? 유료 강의에 130만원 써 봤습니다. 돈 버는 게 그렇게...,Business & Industrial,Business & Industrial/Small Business,Business & Industrial/Small Business/MLM & Bus...,NaN,NaN,NaN,NaN,NaN,NaN
563031,ZzZyzYp8m9w,"[직캠] 191127 지창욱 베트남 인천공항 입국 (jichangwook,池昌旭) ...",Travel,Travel/Air Travel,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 멀티 라벨링

In [15]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()

In [17]:
df_train = df_train.fillna('None')
labels = mlb.fit_transform(df_train.drop(['video_id','title'],1).values)

<ipython-input-17-3c484cef7067>:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  labels = mlb.fit_transform(df_train.drop(['video_id','title'],1).values)


In [18]:
mlb.classes_

array(['Adult', 'Arts & Entertainment',
       'Arts & Entertainment/Celebrities & Entertainment News',
       'Arts & Entertainment/Comics & Animation',
       'Arts & Entertainment/Comics & Animation/Anime & Manga',
       'Arts & Entertainment/Comics & Animation/Cartoons',
       'Arts & Entertainment/Comics & Animation/Comics',
       'Arts & Entertainment/Entertainment Industry',
       'Arts & Entertainment/Entertainment Industry/Film & TV Industry',
       'Arts & Entertainment/Entertainment Industry/Recording Industry',
       'Arts & Entertainment/Events & Listings',
       'Arts & Entertainment/Events & Listings/Bars, Clubs & Nightlife',
       'Arts & Entertainment/Events & Listings/Concerts & Music Festivals',
       'Arts & Entertainment/Events & Listings/Expos & Conventions',
       'Arts & Entertainment/Events & Listings/Film Festivals',
       'Arts & Entertainment/Events & Listings/Movie Listings & Theater Showtimes',
       'Arts & Entertainment/Fun & Trivia',
       

In [81]:
mlb.classes_[405] #405번째 None 존재

'None'

In [66]:
category = np.setdiff1d(mlb.classes_,['None'])
category

array(['Adult', 'Arts & Entertainment',
       'Arts & Entertainment/Celebrities & Entertainment News',
       'Arts & Entertainment/Comics & Animation',
       'Arts & Entertainment/Comics & Animation/Anime & Manga',
       'Arts & Entertainment/Comics & Animation/Cartoons',
       'Arts & Entertainment/Comics & Animation/Comics',
       'Arts & Entertainment/Entertainment Industry',
       'Arts & Entertainment/Entertainment Industry/Film & TV Industry',
       'Arts & Entertainment/Entertainment Industry/Recording Industry',
       'Arts & Entertainment/Events & Listings',
       'Arts & Entertainment/Events & Listings/Bars, Clubs & Nightlife',
       'Arts & Entertainment/Events & Listings/Concerts & Music Festivals',
       'Arts & Entertainment/Events & Listings/Expos & Conventions',
       'Arts & Entertainment/Events & Listings/Film Festivals',
       'Arts & Entertainment/Events & Listings/Movie Listings & Theater Showtimes',
       'Arts & Entertainment/Fun & Trivia',
       

In [85]:
new_labels = np.delete(labels,405,axis=1)
new_labels

array([[0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

### 멀티 라벨링된 테이블로 변환

In [86]:
new_df_train = pd.DataFrame(columns=category, data=new_labels)
new_df_train.insert(0,'title',df_train['title'])
new_df_train.insert(0,'video_id',df_train['video_id'])

In [87]:
new_df_train

,video_id,title,Adult,Arts & Entertainment,Arts & Entertainment/Celebrities & Entertainment News,Arts & Entertainment/Comics & Animation,Arts & Entertainment/Comics & Animation/Anime & Manga,Arts & Entertainment/Comics & Animation/Cartoons,Arts & Entertainment/Comics & Animation/Comics,Arts & Entertainment/Entertainment Industry,...,Travel/Car Rental & Taxi Services,Travel/Cruises & Charters,Travel/Hotels & Accommodations,Travel/Hotels & Accommodations/Vacation Rentals & Short-Term Stays,Travel/Specialty Travel,Travel/Tourist Destinations,Travel/Tourist Destinations/Beaches & Islands,Travel/Tourist Destinations/Mountain & Ski Resorts,Travel/Tourist Destinations/Regional Parks & Gardens,Travel/Tourist Destinations/Theme Parks
0,_-_EByBFBsc,[오토캐드(Autocad) 기초 3D 26강] 경로 돌출(Sweep) 명령어 사용 방법,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,_-_RRUatHAU,[타로 / 연애 / 심리] 나는 누구와 사랑하게 될까?,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,_--c_UgDDsk,갓세븐 제이비랑 울면서 영통한 썰 푼다 (GOT7 JB),0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,_--MzCpmzE4,Black Bean Noodle Song | 창작동요 | 쥬니버 동요,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,_--NaWhpU6E,[방탄소년단 슈가] 이럴슈가..,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
563028,ZZzyk-FkjlE,미니어쳐 빈티지 케이크 만들기/ miniature vintage cake (쪼끄맹 ...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
563029,ZzzyL2yM30Y,"레깅스 입고, 뛰어다니는 아줌마 Vlog",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
563030,ZZzYtQSwo6g,자동 수익으로 월 천만원? 유료 강의에 130만원 써 봤습니다. 돈 버는 게 그렇게...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
563031,ZzZyzYp8m9w,"[직캠] 191127 지창욱 베트남 인천공항 입국 (jichangwook,池昌旭) ...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [88]:
new_df_train.shape

(563033, 584)

### 텍스트 전처리 함수

### 형태소 분석기 Mecab

In [4]:
mecab = Mecab(dicpath='C:/mecab/mecab-ko-dic/')

In [25]:
def cleanText(readData): 
    #텍스트에 포함되어 있는 특수 문자 제거
    text = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', readData)
    return text

In [27]:
SELECT_POS = ['NNG','NNP','XR','SL','SN','VV','VA','ETN','NA','NR','XSN','UNKNOWN'] #선택하고 싶은 품사 선택

In [28]:
def pos_tagging(text):
    word_list = []
    for word in mecab.pos(text):
        if word[1] in SELECT_POS:
            word_list.append(word[0])
    return word_list

In [92]:
new_df_train['title'] = new_df_train['title'].apply(lambda x : cleanText(x))
new_df_train['text'] = new_df_train['title'].apply(lambda x : mecab.morphs(x))
new_df_train = new_df_train.sample(frac=1,random_state=0).reset_index(drop=True) #무작위로 rows 섞기
new_df_train

,video_id,title,Adult,Arts & Entertainment,Arts & Entertainment/Celebrities & Entertainment News,Arts & Entertainment/Comics & Animation,Arts & Entertainment/Comics & Animation/Anime & Manga,Arts & Entertainment/Comics & Animation/Cartoons,Arts & Entertainment/Comics & Animation/Comics,Arts & Entertainment/Entertainment Industry,...,Travel/Cruises & Charters,Travel/Hotels & Accommodations,Travel/Hotels & Accommodations/Vacation Rentals & Short-Term Stays,Travel/Specialty Travel,Travel/Tourist Destinations,Travel/Tourist Destinations/Beaches & Islands,Travel/Tourist Destinations/Mountain & Ski Resorts,Travel/Tourist Destinations/Regional Parks & Gardens,Travel/Tourist Destinations/Theme Parks,text
0,iEuz4J8Qi8Y,초강력 체지방 박살내는 실내자전거 프로그램 65분용 페달링 인터벌,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[초, 강력, 체지방, 박살내, 는, 실내, 자전거, 프로그램, 65, 분용, 페달..."
1,PZNaxTRVZwI,로블록스 오징어게임 또 있다 이게 제일 고퀄,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[로, 블록스, 오징어, 게임, 또, 있, 다, 이게, 제일, 고퀄]"
2,INPMBIXXRtg,거래완료 매물번호 211203 당진토지매매 당진시내권 농지원부가 가능하고 도로...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[거래, 완료, 매물, 번호, 211203, 당진, 토지, 매매, 당진, 시내, 권..."
3,UeFgeKwHIvk,일상vlog 저도 아스트라제네카 백신 맞았습니다 백신과 3일간의 기록 출근길 주행로그,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[일상, vlog, 저, 도, 아스트라, 제, 네, 카, 백신, 맞, 았, 습니다,..."
4,SPU26B2tuOo,EP45 VS탑피오라 럼블 장인이 2ㄷ1 다이브를 막는 법 피오라 상대 법,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[EP, 45, VS, 탑, 피, 오라, 럼블, 장인, 이, 2, ㄷ, 1, 다이브..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
563028,oifkts5KZqI,G90과 함께한 제주여행 서프라이즈 테마파크 완전정복 1억넘는 국내 플래그십세단 자...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[G, 90, 과, 함께, 한, 제주, 여행, 서프라이즈, 테마파크, 완전, 정복,..."
563029,COObuHw0Q0g,주짓수부상 수술 잘 받고 퇴원합니다 반월상연골판파열 베이커낭종 베이커씨낭종 슬와낭...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[주, 짓, 수부, 상, 수술, 잘, 받, 고, 퇴원, 합니다, 반월, 상연, 골판..."
563030,aQVNkgem2b8,소래포구 난전 어시장 생새우 뻘낙지 꽃게 최고 택배 연락처 공개 주문하러 가시죠 K...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[소래포구, 난전, 어시장, 생새우, 뻘, 낙지, 꽃, 게, 최고, 택배, 연락처,..."
563031,SsytN2RcCzA,오헨리 단편선 마지막잎새 크리스마스 선물 책읽어주는여자 오디오북 라디오극장 Ko...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[오헨리, 단편, 선, 마지막, 잎새, 크리스마스, 선물, 책, 읽, 어, 주, 는..."


### Fasttext Embedding SET 불러오기 : Youtube Title 200만개로 학습된 Fasttext Embedding

In [36]:
ft_model = FastText.load('YT_2mil_videos_fasttext.model')

In [37]:
len(ft_model.wv)

192265

In [38]:
#tokenizer 생성
token = Tokenizer()
token.fit_on_texts(new_df_train['text'])
word_index = token.word_index

In [40]:
# save
with open('GCP_train_set_tokenizer.pickle', 'wb') as handle:
    pickle.dump(token, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [93]:
df_train_data = new_df_train[:510000] # 학습 데이터 51만
df_test_data = new_df_train[510000:] # 테스트 데이터 5만

In [94]:
x_train = df_train_data['text']
x_valid = df_test_data['text']

In [95]:
#Y값은 array 형태의 멀티 라벨 
y_train = np.array(df_train_data.iloc[:,2:-1])
y_valid = np.array(df_test_data.iloc[:,2:-1])

### Train의 텍스트를 토큰화 후 Fasttext 임베딩에 적용

In [96]:
Word_vector_size = 100 #100 Size로 학습한 Fasttext이므로 100을 입력

In [97]:
# convert text to sequence of tokens and pad them to ensure equal length vectors 
x_train_seq = sequence.pad_sequences(token.texts_to_sequences(x_train), maxlen=Word_vector_size)
x_valid_seq = sequence.pad_sequences(token.texts_to_sequences(x_valid), maxlen=Word_vector_size)
# create token-embedding mapping
embedding_matrix = np.zeros((len(word_index) + 1, Word_vector_size))
words = []

In [98]:
for word, i in tqdm(word_index.items()):
    embedding_vector = ft_model.wv.get_vector(word) #embeddings_index.get(word)
    words.append(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

100%|███████████████████████████████████████████████████████████████████████| 197241/197241 [00:03<00:00, 61407.55it/s]


In [99]:
embedded = keras.layers.Embedding(len(word_index)+1, 100, weights=[embedding_matrix], trainable=True)

### LSTM 모델 학습

In [100]:
_,n_outputs = y_train.shape

In [101]:
model = Sequential()
model.add(embedded)
model.add(Bidirectional(LSTM(200, return_sequences = True)))
model.add(GlobalMaxPool1D()) #풀링 : 과적합 방지
model.add(Dense(50, activation="relu"))
model.add(Dropout(0.05)) 
model.add(Dense(n_outputs, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [102]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 100)         19724200  
                                                                 
 bidirectional_2 (Bidirectio  (None, None, 400)        481600    
 nal)                                                            
                                                                 
 global_max_pooling1d_2 (Glo  (None, 400)              0         
 balMaxPooling1D)                                                
                                                                 
 dense_4 (Dense)             (None, 50)                20050     
                                                                 
 dropout_2 (Dropout)         (None, 50)                0         
                                                                 
 dense_5 (Dense)             (None, 582)              

In [103]:
history = model.fit(x_train_seq, y_train, epochs=4, batch_size=64,
                    validation_data=(x_valid_seq, y_valid), verbose=1)

Epoch 1/4
7969/7969 [==============================] - 268s 33ms/step - loss: 0.0142 - accuracy: 0.6048 - val_loss: 0.0081 - val_accuracy: 0.6581
Epoch 2/4
7969/7969 [==============================] - 263s 33ms/step - loss: 0.0074 - accuracy: 0.6313 - val_loss: 0.0069 - val_accuracy: 0.6343
Epoch 3/4
7969/7969 [==============================] - 266s 33ms/step - loss: 0.0062 - accuracy: 0.6270 - val_loss: 0.0066 - val_accuracy: 0.5996
Epoch 4/4
7969/7969 [==============================] - 264s 33ms/step - loss: 0.0054 - accuracy: 0.6266 - val_loss: 0.0066 - val_accuracy: 0.6260


In [104]:
model.save('LSTM_TOPIC_FASTTEXT_MultiLabel_220128.h5')

In [ ]:
#model = tf.keras.models.load_model('LSTM_TOPIC_FASTTEXT_MultiLabel_220128.h5')

### TEST DATA 불러오기

In [107]:
host="database-1.cutjb1u2xemu.ap-northeast-2.rds.amazonaws.com"
port=3306
user="dhseo"
passwd="seodh1234"
db="hifen"

In [108]:
conn  = pymysql.connect(host=host,port=port,user=user,passwd=passwd,db=db)  #데이터베이스 연결
cursor = conn.cursor()
conn.set_charset('utf8')

In [109]:
cursor.execute("Select * FROM YT_video_lists") #From 367
columns = [item[0] for item in cursor.description]
data = pd.DataFrame(cursor.fetchall(),columns=columns)

In [110]:
data = data.dropna(subset=['title'])

In [361]:
#Gaming, Comedy, Music은 포함하지 않음
data = data[(data['genre']!='Gaming') & (data['genre']!='Comedy') & (data['genre']!='Music')]

### TEST DATA 전처리

In [362]:
data.reset_index(drop=True,inplace=True)
data['title'] = data['title'].apply(lambda x : cleanText(x))
data['text'] = data['title'].apply(lambda x : mecab.morphs(x))

<ipython-input-362-080c21d5c93b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['title'] = data['title'].apply(lambda x : cleanText(x))
<ipython-input-362-080c21d5c93b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text'] = data['title'].apply(lambda x : mecab.morphs(x))


In [114]:
#LOAD
"""
with open('GCP_train_set_tokenizer.pickle', 'rb') as handle:
    tokeni = pickle.load(handle)
"""

"\nwith open('GCP_train_set_tokenizer.pickle', 'rb') as handle:\n    tokeni = pickle.load(handle)\n"

In [363]:
x_test = data['text']
x_test_seq = sequence.pad_sequences(token.texts_to_sequences(x_test), maxlen=100)

### 모델 추론

In [395]:
y = model.predict(x_test_seq[1000000:])

In [396]:
result = pd.DataFrame(y,columns=df_train_data.iloc[:,2:-1].columns)

In [397]:
test_data = data[1000000:]
test_data.reset_index(drop=True, inplace=True)

In [398]:
result['title'] = test_data['title']
result['video_id'] = test_data['video_id']

In [399]:
result

,Adult,Arts & Entertainment,Arts & Entertainment/Celebrities & Entertainment News,Arts & Entertainment/Comics & Animation,Arts & Entertainment/Comics & Animation/Anime & Manga,Arts & Entertainment/Comics & Animation/Cartoons,Arts & Entertainment/Comics & Animation/Comics,Arts & Entertainment/Entertainment Industry,Arts & Entertainment/Entertainment Industry/Film & TV Industry,Arts & Entertainment/Entertainment Industry/Recording Industry,...,Travel/Hotels & Accommodations,Travel/Hotels & Accommodations/Vacation Rentals & Short-Term Stays,Travel/Specialty Travel,Travel/Tourist Destinations,Travel/Tourist Destinations/Beaches & Islands,Travel/Tourist Destinations/Mountain & Ski Resorts,Travel/Tourist Destinations/Regional Parks & Gardens,Travel/Tourist Destinations/Theme Parks,title,video_id
0,3.535888e-05,0.993365,7.077789e-04,6.964135e-04,4.448527e-04,1.317552e-05,2.704190e-07,2.912351e-04,4.343596e-06,1.957353e-04,...,2.336439e-05,2.930132e-09,1.099322e-09,1.539978e-04,3.416576e-09,4.669571e-11,2.907697e-12,8.914113e-07,As You Wish 유연정 모먼트,mFJ-er5d6Ag
1,6.281564e-06,0.957165,7.129796e-05,1.997687e-03,1.327945e-04,6.560050e-05,4.257636e-07,6.871834e-03,1.096280e-03,7.087715e-04,...,4.217124e-03,3.491963e-04,7.140527e-10,1.366619e-01,9.934854e-05,1.945912e-07,4.408559e-07,2.738830e-04,전세대 복층설계의 프리미엄공간 여수웅천캐슬디아트 가 궁금해,mfj2HwexoTY
2,4.478254e-06,0.272745,1.545973e-08,4.995810e-02,1.820245e-03,9.425518e-04,1.040050e-04,2.582235e-09,5.864469e-10,1.802393e-11,...,1.416327e-06,2.553323e-14,1.698157e-10,1.405614e-06,1.443779e-11,1.832448e-13,4.841829e-13,1.650365e-07,찍어먹는 마쉬멜로우 찍먹파 모여라,mFJ2VlpTwW8
3,2.983851e-03,0.995343,1.011876e-04,2.520877e-02,1.289295e-02,1.281699e-04,1.852926e-05,3.526922e-06,1.847911e-07,1.428340e-07,...,3.015148e-08,2.632590e-11,2.306932e-15,1.271237e-06,4.205380e-12,2.652801e-16,8.001449e-18,3.301948e-09,YOUJIN ONE ChoreographyㅣIggy Azalea Sip It ft...,Mfj6iXkrAQY
4,5.105600e-09,0.056987,9.387482e-13,2.357153e-03,8.365898e-06,3.273428e-05,1.827966e-07,2.108802e-12,1.816011e-13,6.585740e-15,...,5.850853e-09,2.092333e-18,3.015195e-15,1.391408e-08,2.818533e-15,2.965965e-18,3.619661e-16,9.049895e-11,ASMR MUKBANG HONEY JELLY PARTY STRAWBERRY HONE...,mfj8IbMQfUI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
719466,1.236264e-03,0.999255,5.697352e-06,6.915326e-01,7.805229e-02,4.773316e-03,3.703580e-02,7.419653e-06,4.582897e-06,1.008802e-09,...,9.165839e-10,1.497517e-15,4.811333e-19,8.375530e-08,1.469348e-15,1.054582e-18,1.194841e-21,1.062326e-07,Superheroes Dance 슈퍼히어로가 신나게 춤을 춘다고 색칠공부 Hulk ...,ZzZZEbhJOi8
719467,3.092460e-08,0.013229,8.132632e-06,5.329571e-08,4.204043e-11,3.973921e-11,1.120995e-09,7.954667e-08,1.178627e-08,2.935119e-11,...,5.356359e-07,1.034421e-11,1.352533e-22,5.091184e-06,5.867874e-09,6.533609e-10,2.857249e-13,7.749697e-10,프로 골퍼들과 9홀 대결 과연 승자는 feat 시크릿 선물 ㅣ 9 holes aga...,zzzzjjJ69Ew
719468,5.600032e-08,0.160676,8.717512e-09,5.827576e-05,1.137751e-06,2.360907e-07,1.175591e-06,9.159794e-13,5.148431e-13,1.614890e-15,...,3.538234e-08,3.052475e-20,2.340961e-14,4.739432e-05,1.836089e-10,1.611167e-12,1.001990e-11,2.562914e-08,팝잇을 누를 수 있을까요 마인크래프트 탁주 쪼꼬,ZZzzk-BNoJQ
719469,1.577523e-05,0.112371,2.273085e-05,1.062899e-03,4.766427e-04,4.993551e-06,2.604300e-05,4.894296e-04,7.471367e-04,3.381659e-05,...,3.228025e-04,4.289675e-07,7.264762e-07,3.497720e-04,5.698128e-07,2.175735e-07,1.400977e-07,2.852001e-05,파츠소분하고 비즈키트 제작하는 쇼핑몰 브이로그 슬라임 고수님들 도와주세요,zZzZoEbxZK0


In [400]:
filter_col_list = ['Pets & Animals', 'Science', 'Travel', 'Books & Literature','Reference', 'Health', 'Shopping', 'Beauty & Fitness',\
       'Real Estate', 'Finance', 'Business & Industrial', 'Sports','Computers & Electronics', 'Food & Drink', 'Hobbies & Leisure',\
       'Arts & Entertainment', 'Home & Garden', 'People & Society','News', 'Jobs & Education','Games', 'Internet & Telecom',\
        'Law & Government','News','Online Communities','Sensitive Subjects','Adult']

In [401]:
result_filter = result.drop(filter_col_list,1)

<ipython-input-401-10bd6e8f5339>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  result_filter = result.drop(filter_col_list,1)


In [402]:
result_filter

,Arts & Entertainment/Celebrities & Entertainment News,Arts & Entertainment/Comics & Animation,Arts & Entertainment/Comics & Animation/Anime & Manga,Arts & Entertainment/Comics & Animation/Cartoons,Arts & Entertainment/Comics & Animation/Comics,Arts & Entertainment/Entertainment Industry,Arts & Entertainment/Entertainment Industry/Film & TV Industry,Arts & Entertainment/Entertainment Industry/Recording Industry,Arts & Entertainment/Events & Listings,"Arts & Entertainment/Events & Listings/Bars, Clubs & Nightlife",...,Travel/Hotels & Accommodations,Travel/Hotels & Accommodations/Vacation Rentals & Short-Term Stays,Travel/Specialty Travel,Travel/Tourist Destinations,Travel/Tourist Destinations/Beaches & Islands,Travel/Tourist Destinations/Mountain & Ski Resorts,Travel/Tourist Destinations/Regional Parks & Gardens,Travel/Tourist Destinations/Theme Parks,title,video_id
0,7.077789e-04,6.964135e-04,4.448527e-04,1.317552e-05,2.704190e-07,2.912351e-04,4.343596e-06,1.957353e-04,7.124329e-03,2.687397e-04,...,2.336439e-05,2.930132e-09,1.099322e-09,1.539978e-04,3.416576e-09,4.669571e-11,2.907697e-12,8.914113e-07,As You Wish 유연정 모먼트,mFJ-er5d6Ag
1,7.129796e-05,1.997687e-03,1.327945e-04,6.560050e-05,4.257636e-07,6.871834e-03,1.096280e-03,7.087715e-04,9.166499e-03,7.382202e-05,...,4.217124e-03,3.491963e-04,7.140527e-10,1.366619e-01,9.934854e-05,1.945912e-07,4.408559e-07,2.738830e-04,전세대 복층설계의 프리미엄공간 여수웅천캐슬디아트 가 궁금해,mfj2HwexoTY
2,1.545973e-08,4.995810e-02,1.820245e-03,9.425518e-04,1.040050e-04,2.582235e-09,5.864469e-10,1.802393e-11,5.425446e-06,1.242601e-06,...,1.416327e-06,2.553323e-14,1.698157e-10,1.405614e-06,1.443779e-11,1.832448e-13,4.841829e-13,1.650365e-07,찍어먹는 마쉬멜로우 찍먹파 모여라,mFJ2VlpTwW8
3,1.011876e-04,2.520877e-02,1.289295e-02,1.281699e-04,1.852926e-05,3.526922e-06,1.847911e-07,1.428340e-07,2.945932e-04,1.189432e-06,...,3.015148e-08,2.632590e-11,2.306932e-15,1.271237e-06,4.205380e-12,2.652801e-16,8.001449e-18,3.301948e-09,YOUJIN ONE ChoreographyㅣIggy Azalea Sip It ft...,Mfj6iXkrAQY
4,9.387482e-13,2.357153e-03,8.365898e-06,3.273428e-05,1.827966e-07,2.108802e-12,1.816011e-13,6.585740e-15,6.221222e-08,1.892800e-09,...,5.850853e-09,2.092333e-18,3.015195e-15,1.391408e-08,2.818533e-15,2.965965e-18,3.619661e-16,9.049895e-11,ASMR MUKBANG HONEY JELLY PARTY STRAWBERRY HONE...,mfj8IbMQfUI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
719466,5.697352e-06,6.915326e-01,7.805229e-02,4.773316e-03,3.703580e-02,7.419653e-06,4.582897e-06,1.008802e-09,4.668515e-06,5.796655e-10,...,9.165839e-10,1.497517e-15,4.811333e-19,8.375530e-08,1.469348e-15,1.054582e-18,1.194841e-21,1.062326e-07,Superheroes Dance 슈퍼히어로가 신나게 춤을 춘다고 색칠공부 Hulk ...,ZzZZEbhJOi8
719467,8.132632e-06,5.329571e-08,4.204043e-11,3.973921e-11,1.120995e-09,7.954667e-08,1.178627e-08,2.935119e-11,5.214786e-09,7.921765e-12,...,5.356359e-07,1.034421e-11,1.352533e-22,5.091184e-06,5.867874e-09,6.533609e-10,2.857249e-13,7.749697e-10,프로 골퍼들과 9홀 대결 과연 승자는 feat 시크릿 선물 ㅣ 9 holes aga...,zzzzjjJ69Ew
719468,8.717512e-09,5.827576e-05,1.137751e-06,2.360907e-07,1.175591e-06,9.159794e-13,5.148431e-13,1.614890e-15,1.905541e-07,1.391394e-09,...,3.538234e-08,3.052475e-20,2.340961e-14,4.739432e-05,1.836089e-10,1.611167e-12,1.001990e-11,2.562914e-08,팝잇을 누를 수 있을까요 마인크래프트 탁주 쪼꼬,ZZzzk-BNoJQ
719469,2.273085e-05,1.062899e-03,4.766427e-04,4.993551e-06,2.604300e-05,4.894296e-04,7.471367e-04,3.381659e-05,1.256545e-03,1.482969e-04,...,3.228025e-04,4.289675e-07,7.264762e-07,3.497720e-04,5.698128e-07,2.175735e-07,1.400977e-07,2.852001e-05,파츠소분하고 비즈키트 제작하는 쇼핑몰 브이로그 슬라임 고수님들 도와주세요,zZzZoEbxZK0


In [403]:
df_result_filter = pd.DataFrame(np.where(result_filter.iloc[:,:-2]<0.5,0,result_filter.iloc[:,:-2]),columns = result_filter.iloc[:,:-2].columns)

In [404]:
highest_value_category = df_result_filter.apply(lambda s: s.nlargest(3).index.tolist(), axis=1)

In [334]:
#highest_value_category = result_filter.iloc[:,:-2].apply(lambda s: s.nlargest(3).index.tolist(), axis=1)

In [405]:
highest_value_category

0         [Arts & Entertainment/Music & Audio, Arts & En...
1         [Arts & Entertainment/Visual Art & Design, Art...
2         [Food & Drink/Food, Arts & Entertainment/Celeb...
3         [Arts & Entertainment/Performing Arts, Arts & ...
4         [Food & Drink/Food, Arts & Entertainment/Celeb...
                                ...                        
719466    [Arts & Entertainment/Comics & Animation, Arts...
719467    [Sports/Individual Sports, Sports/Individual S...
719468    [Games/Computer & Video Games, Arts & Entertai...
719469    [Arts & Entertainment/Celebrities & Entertainm...
719470    [Arts & Entertainment/Celebrities & Entertainm...
Length: 719471, dtype: object

In [406]:
highest_value_score = df_result_filter.apply(lambda s: s.nlargest(3).tolist(), axis=1)

In [337]:
#highest_value_score = result_filter.iloc[:,:-2].apply(lambda s: s.nlargest(3).tolist(), axis=1)

In [376]:
highest_value_score

0                            [0.7496365308761597, 0.0, 0.0]
1                            [0.7474111318588257, 0.0, 0.0]
2                            [0.9428663849830627, 0.0, 0.0]
3                                           [0.0, 0.0, 0.0]
4                            [0.9968382120132446, 0.0, 0.0]
                                ...                        
999995                       [0.8870100378990173, 0.0, 0.0]
999996                       [0.8178060054779053, 0.0, 0.0]
999997                       [0.7383327484130859, 0.0, 0.0]
999998    [0.983829915523529, 0.976870059967041, 0.96610...
999999                                      [0.0, 0.0, 0.0]
Length: 1000000, dtype: object

In [407]:
df_results = pd.DataFrame(dict(category = highest_value_category, score = highest_value_score)).reset_index(drop=True)
df_results

,category,score
0,"[Arts & Entertainment/Music & Audio, Arts & En...","[0.9790673851966858, 0.0, 0.0]"
1,"[Arts & Entertainment/Visual Art & Design, Art...","[0.9078359007835388, 0.0, 0.0]"
2,"[Food & Drink/Food, Arts & Entertainment/Celeb...","[0.8201890587806702, 0.0, 0.0]"
3,"[Arts & Entertainment/Performing Arts, Arts & ...","[0.8160056471824646, 0.671584963798523, 0.0]"
4,"[Food & Drink/Food, Arts & Entertainment/Celeb...","[0.9156589508056641, 0.0, 0.0]"
...,...,...
719466,"[Arts & Entertainment/Comics & Animation, Arts...","[0.691532552242279, 0.500653862953186, 0.0]"
719467,"[Sports/Individual Sports, Sports/Individual S...","[0.9967355132102966, 0.9398913383483887, 0.0]"
719468,"[Games/Computer & Video Games, Arts & Entertai...","[0.9857289791107178, 0.0, 0.0]"
719469,[Arts & Entertainment/Celebrities & Entertainm...,"[0.0, 0.0, 0.0]"


In [408]:
cate_result = pd.DataFrame(df_results,columns=['category'])
objs = [cate_result, pd.DataFrame(cate_result['category'].tolist())]
df_category = pd.concat(objs, axis=1).drop('category', axis=1)
df_category.columns=['main_category','sub1_category','sub2_category']
df_category

,main_category,sub1_category,sub2_category
0,Arts & Entertainment/Music & Audio,Arts & Entertainment/Celebrities & Entertainme...,Arts & Entertainment/Comics & Animation
1,Arts & Entertainment/Visual Art & Design,Arts & Entertainment/Celebrities & Entertainme...,Arts & Entertainment/Comics & Animation
2,Food & Drink/Food,Arts & Entertainment/Celebrities & Entertainme...,Arts & Entertainment/Comics & Animation
3,Arts & Entertainment/Performing Arts,Arts & Entertainment/Performing Arts/Dance,Arts & Entertainment/Celebrities & Entertainme...
4,Food & Drink/Food,Arts & Entertainment/Celebrities & Entertainme...,Arts & Entertainment/Comics & Animation
...,...,...,...
719466,Arts & Entertainment/Comics & Animation,Arts & Entertainment/Performing Arts,Arts & Entertainment/Celebrities & Entertainme...
719467,Sports/Individual Sports,Sports/Individual Sports/Golf,Arts & Entertainment/Celebrities & Entertainme...
719468,Games/Computer & Video Games,Arts & Entertainment/Celebrities & Entertainme...,Arts & Entertainment/Comics & Animation
719469,Arts & Entertainment/Celebrities & Entertainme...,Arts & Entertainment/Comics & Animation,Arts & Entertainment/Comics & Animation/Anime ...


In [409]:
cate_result = pd.DataFrame(df_results,columns=['score'])
objs = [cate_result, pd.DataFrame(cate_result['score'].tolist())]

In [410]:
df_score = pd.concat(objs, axis=1).drop('score', axis=1)
df_score.columns=['main_score','sub1_score','sub2_score']
df_score

,main_score,sub1_score,sub2_score
0,0.979067,0.000000,0.0
1,0.907836,0.000000,0.0
2,0.820189,0.000000,0.0
3,0.816006,0.671585,0.0
4,0.915659,0.000000,0.0
...,...,...,...
719466,0.691533,0.500654,0.0
719467,0.996736,0.939891,0.0
719468,0.985729,0.000000,0.0
719469,0.000000,0.000000,0.0


In [411]:
df_results2 = pd.concat([df_category,df_score],axis=1)

In [412]:
df_results2.loc[df_results2['main_score'] == 0, 'main_category'] = ''
df_results2.loc[df_results2['sub1_score'] == 0, 'sub1_category'] = ''
df_results2.loc[df_results2['sub2_score'] == 0, 'sub2_category'] = ''

In [413]:
df_results2

,main_category,sub1_category,sub2_category,main_score,sub1_score,sub2_score
0,Arts & Entertainment/Music & Audio,,,0.979067,0.000000,0.0
1,Arts & Entertainment/Visual Art & Design,,,0.907836,0.000000,0.0
2,Food & Drink/Food,,,0.820189,0.000000,0.0
3,Arts & Entertainment/Performing Arts,Arts & Entertainment/Performing Arts/Dance,,0.816006,0.671585,0.0
4,Food & Drink/Food,,,0.915659,0.000000,0.0
...,...,...,...,...,...,...
719466,Arts & Entertainment/Comics & Animation,Arts & Entertainment/Performing Arts,,0.691533,0.500654,0.0
719467,Sports/Individual Sports,Sports/Individual Sports/Golf,,0.996736,0.939891,0.0
719468,Games/Computer & Video Games,,,0.985729,0.000000,0.0
719469,,,,0.000000,0.000000,0.0


In [414]:
df_results2['title'] = result['title']
df_results2['video_id'] = result['video_id']

In [415]:
df_results2

,main_category,sub1_category,sub2_category,main_score,sub1_score,sub2_score,title,video_id
0,Arts & Entertainment/Music & Audio,,,0.979067,0.000000,0.0,As You Wish 유연정 모먼트,mFJ-er5d6Ag
1,Arts & Entertainment/Visual Art & Design,,,0.907836,0.000000,0.0,전세대 복층설계의 프리미엄공간 여수웅천캐슬디아트 가 궁금해,mfj2HwexoTY
2,Food & Drink/Food,,,0.820189,0.000000,0.0,찍어먹는 마쉬멜로우 찍먹파 모여라,mFJ2VlpTwW8
3,Arts & Entertainment/Performing Arts,Arts & Entertainment/Performing Arts/Dance,,0.816006,0.671585,0.0,YOUJIN ONE ChoreographyㅣIggy Azalea Sip It ft...,Mfj6iXkrAQY
4,Food & Drink/Food,,,0.915659,0.000000,0.0,ASMR MUKBANG HONEY JELLY PARTY STRAWBERRY HONE...,mfj8IbMQfUI
...,...,...,...,...,...,...,...,...
719466,Arts & Entertainment/Comics & Animation,Arts & Entertainment/Performing Arts,,0.691533,0.500654,0.0,Superheroes Dance 슈퍼히어로가 신나게 춤을 춘다고 색칠공부 Hulk ...,ZzZZEbhJOi8
719467,Sports/Individual Sports,Sports/Individual Sports/Golf,,0.996736,0.939891,0.0,프로 골퍼들과 9홀 대결 과연 승자는 feat 시크릿 선물 ㅣ 9 holes aga...,zzzzjjJ69Ew
719468,Games/Computer & Video Games,,,0.985729,0.000000,0.0,팝잇을 누를 수 있을까요 마인크래프트 탁주 쪼꼬,ZZzzk-BNoJQ
719469,,,,0.000000,0.000000,0.0,파츠소분하고 비즈키트 제작하는 쇼핑몰 브이로그 슬라임 고수님들 도와주세요,zZzZoEbxZK0


In [416]:
df_results2.to_csv('./result_test_rest(0.5cut).csv',encoding='utf-8-sig',index=False)

In [417]:
insert_df = df_results2[df_results2['main_score']>0].drop(['title'],1)

<ipython-input-417-ca40aaa006b8>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  insert_df = df_results2[df_results2['main_score']>0].drop(['title'],1)


In [418]:
insert_df

,main_category,sub1_category,sub2_category,main_score,sub1_score,sub2_score,video_id
0,Arts & Entertainment/Music & Audio,,,0.979067,0.000000,0.0,mFJ-er5d6Ag
1,Arts & Entertainment/Visual Art & Design,,,0.907836,0.000000,0.0,mfj2HwexoTY
2,Food & Drink/Food,,,0.820189,0.000000,0.0,mFJ2VlpTwW8
3,Arts & Entertainment/Performing Arts,Arts & Entertainment/Performing Arts/Dance,,0.816006,0.671585,0.0,Mfj6iXkrAQY
4,Food & Drink/Food,,,0.915659,0.000000,0.0,mfj8IbMQfUI
...,...,...,...,...,...,...,...
719463,Business & Industrial/Small Business,Business & Industrial/Small Business/MLM & Bus...,,0.967030,0.966898,0.0,ZZzYtQSwo6g
719464,Travel/Air Travel,,,0.944208,0.000000,0.0,ZzZyzYp8m9w
719466,Arts & Entertainment/Comics & Animation,Arts & Entertainment/Performing Arts,,0.691533,0.500654,0.0,ZzZZEbhJOi8
719467,Sports/Individual Sports,Sports/Individual Sports/Golf,,0.996736,0.939891,0.0,zzzzjjJ69Ew


### DataBase에 입력

In [393]:
from sqlalchemy import create_engine
from sqlalchemy.ext.compiler import compiles
from sqlalchemy.sql.expression import Insert
from sqlalchemy import create_engine
pymysql.install_as_MySQLdb()



In [419]:
engine = create_engine("mysql+mysqldb://"+user+":"+str(passwd)+"@"+host+":"+str(port)+"/"+db, encoding='utf-8')
insert_df.to_sql(name='YT_video_category_class', con=engine, if_exists = 'append', index=False)    